#### 通用模块

In [1]:
import pandas as pd
import re
import numpy as np
def get_dir_filenames(path,ftype):
    '''获取目录下所有指定格式的文件名'''
    import os
    names = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if ftype in file:
                names.append(file)
    return names


#### 论文数据统计

In [2]:
def get_paper_total(source_path,subject,target_path):
    '''source_path:数据来源
       subject:学科
       target_path:导出的文件名'''
    df1 = pd.read_excel(source_path)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    col_list = {}
    for item in cols:
        col_list[item] = []
    #拆分学校
    for index,row in df1.iterrows():
        for col in cols:   
            if col in row['Organ-单位']:
                col_list[col].append(row)
    temp_list = []
    for col in col_list:
        counts = len(col_list[col])
        row = {'学校名称':col,'论文数':counts,'学科':subject}
        temp_list.append(row)
    df2 = pd.DataFrame(temp_list)
    df2.to_excel(target_path,index=False)
    print('finish')

def get_paper_detail(source_path,subject,target_path):
    '''source_path:数据来源
       subject:学科
       target_path:要存放数据的目录'''
    df1 = pd.read_excel(source_path)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    col_list = {}
    for item in cols:
        col_list[item] = []
    #拆分学校
    for index,row in df1.iterrows():
        for col in cols:   
            if col in row['Organ-单位']:
                col_list[col].append(row)
    years = ['2016','2017','2018','2019']
    all_list = []
    for col in col_list:
        counts = {}
        for item in years:
            counts[item] = 0
        df2 = pd.DataFrame(col_list[col])
        for index,row in df2.iterrows():
            for year in years:
                if year == str(row['Year-年']):
                    counts[year]+=1
                    break
        temp_list = []
        for year in counts:
            row = {'学校名称':col,'发表时间':year,'论文数':counts[year]}
            temp_list.append(row)
        df3 = pd.DataFrame(temp_list)
        all_list.append(df3)
    df4 = pd.concat(all_list)
    mdict = {}
    for item in years:
        mdict[item] = []
    for index,row in df4.iterrows():
        mdict[str(row['发表时间'])].append(row)
    for item in mdict:
        df5 = pd.DataFrame(mdict[item])
        df5.to_excel(target_path+'/'+subject+'-'+item+'-中文期刊论文.xlsx',index=False)
    print('finish')
    
#get_paper_detail('中文期刊论文/统计学汇总.xlsx','统计学','test')

In [4]:
get_paper_total('中文期刊论文/法学汇总.xlsx','法学','报表数据/法学/2016-2019年各校中文期刊论文总体情况.xlsx')
get_paper_detail('中文期刊论文/法学汇总.xlsx','法学','报表数据/法学/中文期刊论文')

finish
finish


#### 国家社科项目总数统计和各年度数据统计
如果当年没有项目，需要明确指定为零

In [21]:
def get_sk_total(source_path,subject,target_path):
    '''source_path:数据来源
       subject:学科
       target_path:导出的文件名'''
    df = pd.read_excel(source_path)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    temp_list = []
    for index,row in df.iterrows():
        if row['工作单位'].strip() in cols:
            temp_list.append(row)
    df = pd.DataFrame(temp_list) # 获取相关的学校
    temp =pd.DataFrame(df['工作单位'].value_counts())

    temp['学科'] = subject
    temp.to_excel(target_path)
    print('get_total() has finished.')

def get_sk_detail(source_path,subject,target_path):
    '''source_path:数据来源
       subject:学科
       target_path:要存放数据的目录'''
    # 获取相关的学校
    df1 = pd.read_excel(source_path)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    temp_list = []
    for index,row in df1.iterrows():
        if row['工作单位'].strip() in cols:
            temp_list.append(row)
    df2 = pd.DataFrame(temp_list) 
    #拆分各校
    cols = [x.strip() for x in df2['工作单位']]
    col_dic = {}
    for item in cols:
        col_dic[item] = []
    for index, row in df2.iterrows():
        col_dic[row['工作单位']].append(row)
    #统计各校各年度数据
    years = ['2016','2017','2018','2019']
    all_list = []
    #fnames = get_dir_filenames('报表数据/应用经济学/国家社科-各校','.xlsx')
    for key in col_dic:
        counts = {}
        for item in years:
            counts[item] = 0
        df3 = pd.DataFrame(col_dic[key])
        for index, row in df3.iterrows():
            for year in years:
                if str(year) == str(row['立项时间'])[:4]:
                    counts[year]+=1
                    break
        temp_list = []
        for year in counts:
            row = {'学校名称':key,'立项时间':year,'项目数':counts[year]}
            temp_list.append(row)
        df4 = pd.DataFrame(temp_list)
        all_list.append(df4)
    df5 = pd.concat(all_list)
    mdict = {}
    for item in years:
        mdict[item] = []
    for index,row in df5.iterrows():
        mdict[str(row['立项时间'])].append(row)
    for item in mdict:
        df6 = pd.DataFrame(mdict[item])
        df6.to_excel(target_path+'/'+subject+'-'+item+'-国社科.xlsx',index=False)
    print('finish')

In [17]:
get_sk_total('国家社科项目/国家社科-统计学.xlsx','统计学','报表数据/统计学/2016-2019年各校国家社会科学基金项目数总体情况.xlsx')
get_sk_detail('国家社科项目/国家社科-统计学.xlsx','统计学','报表数据/统计学/国家社科')

get_total() has finished.
finish


finish


#### 国家自然科学基金

In [18]:
def get_zr_total(dpath,subject,spath):
    '''
    dpath:原始数据目录
    spath:导出的数据文件名'''
    fnames = get_dir_filenames(dpath,'.xlsx')
    temp_list = []
    for name in fnames:
        df1 = pd.read_excel(dpath+'/'+name)
        row = {'学校名称':name.split('.')[0],'学科':subject,'项目数':len(df1)}
        temp_list.append(row)
    df2 = pd.DataFrame(temp_list)
    df2.to_excel(spath,index=False)
    print('finish')

def get_zr_detail(dpath,subject,spath):
    '''
    dpath:原始数据目录
    spath:导出的目录'''
    fnames = get_dir_filenames(dpath,'.xlsx')
    years = ['2016','2017','2018','2019']
    all_list = []
    for name in fnames:
        df1 = pd.read_excel(dpath+'/'+name)
        count_dict = {}
        for year in years:
            count_dict[year] = 0
        for index, row in df1.iterrows():
            for year in count_dict:
                if year == str(row['批准年份']):
                    count_dict[year]+=1
        temp_list = []
        for year in count_dict:
            row = {'学校名称':name.split('.')[0],'批准年份':str(year),'学科':subject,'项目数':count_dict[year]}
            temp_list.append(row)
        df2 = pd.DataFrame(temp_list)
        all_list.append(df2)
    df3 = pd.concat(all_list)
    mdict = {}
    for item in years:
        mdict[item] = []
    for index,row in df3.iterrows():
        mdict[str(row['批准年份'])].append(row)
    for item in mdict:
        df4 = pd.DataFrame(mdict[item])
        df4.to_excel(spath+'/'+subject+'-'+item+'-国自然.xlsx',index=False)
    print('finish')
    

In [19]:
# get_zr_total('国家自然项目/统计学','统计学','报表数据/统计学/2016-2019年国家自然科学基金项目数总体情况.xlsx')
# get_zr_detail('国家自然项目/统计学','统计学','报表数据/统计学/国家自然')

finish
finish


finish


#### 教育部项目

In [20]:
def get_jyb_total(dpath,subject,spath):
    df1 = pd.read_excel(dpath)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    col_list = {}
    for item in cols:
        col_list[item] = []
    #拆分学校
    for index,row in df1.iterrows():
        col_list[row['学校名称']].append(row)
    temp_list = []
    for col in col_list:
        counts = len(col_list[col])
        row = {'学校名称':col,'项目数':counts,'学科':subject}
        temp_list.append(row)
    df2 = pd.DataFrame(temp_list)
    df2.to_excel(spath,index=False)
    print('finish')
#get_jyb_total('教育部项目/应用经济学.xlsx','应用经济学','报表数据/应用经济学/2016-2019年各校教育部项目总体情况.xlsx')

def get_jyb_detail(dpath,subject,spath):
    # 获取相关的学校
    df1 = pd.read_excel(dpath)
    cols = pd.read_excel('基础数据/数据采集名单.xlsx',sheet_name=subject)
    cols = [x.strip() for x in cols['学校名称']]
    temp_list = []
    for index,row in df1.iterrows():
        if row['学校名称'].strip() in cols:
            temp_list.append(row)
    df2 = pd.DataFrame(temp_list) 
    #拆分各校
    cols = [x.strip() for x in df2['学校名称']]
    col_dic = {}
    for item in cols:
        col_dic[item] = []
    for index, row in df2.iterrows():
        col_dic[row['学校名称']].append(row)
    #统计各校各年度数据
    years = ['2016','2017','2018','2019']
    all_list = []
    #fnames = get_dir_filenames('报表数据/应用经济学/国家社科-各校','.xlsx')
    for key in col_dic:
        counts = {}
        for item in years:
            counts[item] = 0
        df3 = pd.DataFrame(col_dic[key])
        for index, row in df3.iterrows():
            for year in years:
                if str(year) == str(row['立项时间'])[:4]:
                    counts[year]+=1
                    break
        temp_list = []
        for year in counts:
            row = {'学校名称':key,'立项时间':year,'项目数':counts[year]}
            temp_list.append(row)
        df4 = pd.DataFrame(temp_list)
        all_list.append(df4)
    df5 = pd.concat(all_list)
    mdict = {}
    for item in years:
        mdict[item] = []
    for index,row in df5.iterrows():
        mdict[str(row['立项时间'])].append(row)
    for item in mdict:
        df6 = pd.DataFrame(mdict[item])
        df6.to_excel(spath+'/'+subject+'-'+item+'-教育部.xlsx',index=False)
    print('finish')
#get_jyb_detail('教育部项目/应用经济学.xlsx','应用经济学','报表数据/应用经济学')

In [22]:
get_jyb_total('教育部项目/统计学.xlsx','统计学','报表数据/统计学/2016-2019年各校教育部项目数总体情况.xlsx')
get_jyb_detail('教育部项目/统计学.xlsx','统计学','报表数据/统计学/教育部')

finish
finish


finish


#### 中文论文关键词云统计

In [15]:
def get_years_wordcloud(dpath,spath):
    df = pd.read_excel(dpath)
    items = [x for x in df['Keyword-关键词']]
    all_list = []
    for item in items:
        try:
            all_list += item.split(';;')
        except:
            print('kw is null')
            continue
    kws = set(all_list)
    list_dict = []
    for item in kws:
        row = {'关键词':item, '个数':all_list.count(item)}
        list_dict.append(row)
    df1 = pd.DataFrame(list_dict)
    df2 = df1.nlargest(100,'个数')
    mdict = df2.to_dict(orient='records')
    mdict = str(mdict).replace('},','},\n').replace('\'','"')
    with open(spath,'w',encoding='utf-8') as f:
        f.write(mdict)
    print('finish')
    
#get_years_wordcloud('中文期刊论文/应用经济学汇总.xlsx','报表数据/应用经济学/应用经济学词云.txt')
def get_every_year_wc(dpath,spath):
    df = pd.read_excel(dpath)
    df1 = pd.DataFrame()
    df1['关键词'] = df['Keyword-关键词']
    df1['发表年份'] = df['Year-年']
    #按年份拆分论文
    years = ['2016','2017','2018','2019']
    mdict_0 = {}
    for year in years:
        mdict_0[year] = []
    for index,row in df1.iterrows():
#         try:
        mdict_0[str(row['发表年份'])].append(row)
#         except:
#             print('Row {} maybe null.'.format(index))
    #计算各年度关键词数
    for year in years:
        df2 = pd.DataFrame(mdict_0[year])
        try:
            items = [x.strip() for x in df2['关键词']]
        except:
            print('kw is null')
        all_list = []
        for item in items:
            try:
                all_list += item.split(';;')
            except:
                print('kw is null')
                continue
        kws = set(all_list)
        list_dict = []
        for item in kws:
            row = {'关键词':item, '个数':all_list.count(item)}
            list_dict.append(row)
        df3 = pd.DataFrame(list_dict)
        df4 = df3.nlargest(100,'个数')
        mdict = df4.to_dict(orient='records')
        mdict = str(mdict).replace('},','},\n').replace('\'','"')
        with open(spath+'/'+year+'-关键词云.txt','w',encoding='utf-8') as f:
            f.write(mdict)
        print( year+'-关键词云.txt finish')


In [16]:
get_years_wordcloud('中文期刊论文/法学汇总.xlsx','临时数据/2016-2019年统计学-关键词云.txt')
get_every_year_wc('中文期刊论文/法学汇总.xlsx','临时数据')

kw is null
kw is null
finish
2016-关键词云.txt finish
2017-关键词云.txt finish
kw is null
2018-关键词云.txt finish
2019-关键词云.txt finish


In [19]:
# get_every_year_wc('中文期刊论文/工商管理汇总.xlsx','报表数据/工商管理/关键词云')

2016-关键词云.txt finish
2017-关键词云.txt finish
kw is null
2018-关键词云.txt finish
2019-关键词云.txt finish


#### 国家自然科学基金项目

### json 数据生成

#### 汇总数据2json

In [6]:
def all_to_json(dpath,spath,sorted_item):
    df = pd.read_excel(dpath)
    df = df.sort_values(by=sorted_item,ascending=False)
    temp_list = df.to_dict(orient='records')
    mdict = {}
    mdict['datas'] = temp_list
    title = [{"text":spath.split('/')[-1].split('.')[0],"ge":"访问来源"}]
    mdict['title'] = title
    mdict = str(mdict).replace('},','},\n').replace('\'','"')
    with open(spath,'w',encoding='utf-8') as f:
        f.write(mdict)
    print('finish')
    
def every_year2json(dpath,spath):
    '''各年度具体数据
    dpath:原始数据路径
    spath:导出数据路径'''
    fnames = get_dir_filenames(dpath,'.xlsx')
    for name in fnames:
        df = pd.read_excel(dpath+'/'+name)
        mdict = df.to_dict(orient='records')
        mdict = str(mdict).replace('},','},\n').replace('\'','"')
        with open(spath+'/'+name.split('.')[0].split('-')[1]+ '.txt','w',encoding='utf-8') as f:
            f.write(mdict)
    print('finish')
    
def every_year2json_sort(dpath,spath,sorted_item):
    '''各年度具体数据
    dpath:原始数据路径
    spath:导出数据路径'''
    fnames = get_dir_filenames(dpath,'.xlsx')
    for name in fnames:
        df = pd.read_excel(dpath+'/'+name)
        df = df.sort_values(by=sorted_item,ascending=False)
        mdict = df.to_dict(orient='records')
        mdict = str(mdict).replace('},','},\n').replace('\'','"')
        with open(spath+'/'+name.split('.')[0].split('-')[1]+ '-sorted.txt','w',encoding='utf-8') as f:
            f.write(mdict)
    print('finish')

In [11]:
fname = '2016-2019年各校中文期刊论文总体情况'
all_to_json('报表数据/法学/{}.xlsx'.format(fname),'临时数据/{}.json'.format(fname),'论文数')

finish


In [14]:
every_year2json('报表数据/法学/中文期刊论文','临时数据')
every_year2json_sort('报表数据/法学/中文期刊论文','临时数据','论文数')

finish
finish


#### 历次学科评估结果-json

In [9]:
fnames = get_dir_filenames('历次全国学科评估结果/前三轮','.xlsx')
for name in fnames:
    df = pd.read_excel('历次全国学科评估结果/前三轮/'+name)
    cols = [x.strip() for x in df['学校名称']]
    rank = [x+1 for x in np.arange(len(df))]
    df1 = pd.DataFrame()
    df1['学校名称'] = cols
    df1['排名'] = rank
    mdict = df1.to_dict(orient='records')
    mdict = str(mdict).replace('},','},\n').replace('\'','"').replace('"学校名称"','"department"').replace('"排名"','"rank"')
    with open('历次全国学科评估结果/json/'+name.split('.')[0]+'.json','w',encoding='utf-8') as f:
        f.write(mdict)

In [12]:
fnames = get_dir_filenames('历次全国学科评估结果/第四轮','.xlsx')
for name in fnames:
    df = pd.read_excel('历次全国学科评估结果/第四轮/'+name)
    cols = [re.split('\s+',x.strip())[1] for x in df['学校代码及名称']]
    rank = [x.strip() for x in df['评估结果']]
    df1 = pd.DataFrame()
    df1['department'] = cols
    df1['rank'] = rank
    mdict = df1.to_dict(orient='records')
    mdict = str(mdict).replace('},','},\n').replace('\'','"')
    with open('历次全国学科评估结果/json/'+name.split('.')[0]+'.json','w',encoding='utf-8') as f:
        f.write(mdict)

In [4]:
rank

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]